# Experiment Parameters

In [1]:
dino_dataset_names = ["headlines-naive-7600-dataset.jsonl", "headlines-processed-human-dataset.jsonl"]
model_name = "bert-base-uncased" # "distilbert-base-uncased"
num_examples = 6200
shrink_eval = -1
batch_sz = 16
num_epochs = 2
num_evals_per_epoch = 3
learning_rate = 2e-4

# Installs

In [2]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Git

In [3]:
!git clone https://github.com/jasony123123/dino

fatal: destination path 'dino' already exists and is not an empty directory.


In [4]:
%cd dino/datasets

/content/dino/datasets


# Load Datasets

In [5]:
from datasets import load_dataset

real_dataset = load_dataset("ag_news")

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
import json
import datasets

def read_jsonl_dataset(filename):
  train_dataset = datasets.load_dataset("json", data_files=filename)
  labels = {"world": 0, "sports": 1, "business": 2, "scitech": 3}
  mapping = lambda example: {"text": example["text_a"], "label": labels[example["label"]]}

  train_dataset = train_dataset.map(mapping, remove_columns=["text_a","text_b"])
  final_dataset = datasets.DatasetDict({"train": train_dataset["train"]})
  return final_dataset

dino_datasets = [(read_jsonl_dataset(fname), fname) for fname in dino_dataset_names]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_real_datasets = real_dataset.map(tokenize_function, batched=True)
tokenized_dino_datasets = [
    (data.map(tokenize_function, batched=True), name)
    for data, name in dino_datasets
]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

# Experiment

In [8]:
small_eval_real_dataset = tokenized_real_datasets["test"].shuffle(seed=42)
if shrink_eval > 0:
  small_eval_real_dataset = small_eval_real_dataset.select(range(shrink_eval))

null_real_dataset = tokenized_real_datasets["train"].shuffle(seed=42).select(range(1))
small_train_real_dataset = tokenized_real_datasets["train"].shuffle(seed=42).select(range(num_examples))
small_train_dino_datasets = [
    (data["train"].shuffle(seed=42).select(range(num_examples)), name)
    for data, name in tokenized_dino_datasets
]

In [9]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

eval_steps = (num_examples / batch_sz) // num_evals_per_epoch
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="steps",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_sz,
    per_device_eval_batch_size=batch_sz,
    learning_rate=learning_rate,
    eval_steps=eval_steps,
    logging_strategy="steps",
    logging_steps=eval_steps,
    report_to="none",
)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
training_datasets = [
    # (null_real_dataset, "none"),
    (small_train_real_dataset, "real"),
    *small_train_dino_datasets
]

for train_dataset, name in training_datasets:
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=small_eval_real_dataset,
      compute_metrics=compute_metrics,
  )

  print("------------------")
  print(name)
  print("------------------")
  trainer.train()

------------------
real
------------------


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


<a id='pytorch_native'></a>

<a id='additional-resources'></a>